In [1]:
import re
import math

In [2]:
def field_check():
    while True:
        user_input = input(
        f'''Welcome to Auto Driving Car Simulation!
        \nPlease enter the width and height of the simulation field in x y format:\n''')
        
        # Regular expression to match exactly two integers separated by space
        if re.fullmatch(r"\d+ \d+", user_input):
            
            # Split the input and convert to integers
            num1, num2 = map(int, user_input.split())
            
            # Check if both integers are within the specified range  
            print(f"You have created a field of {num1} x {num2}.")
            return num1, num2
        else:
            print("Invalid input. Please enter exactly two integers separated by a space.")


In [3]:
def main_menu():

    while True:
        result = input('''
        \nPlease choose from the following options:
        \n[1] Add a car to field
        \n[2] Run simulation\n''').strip()

        if result.isdigit() and (int(result) == 1 or int(result) == 2) :
            return int(result)
        else:
            print("\nPlease choose either [1] or [2] only")

# main_menu()

In [4]:
def car_start(num1,num2,data):

    while True:
        car_name = input("Please input the name of the car:\n").strip().upper()
        if car_name in [a[0] for a in data]:
            overwrite_check = input("This name already exists, do you want to overwrite? Y/N").strip().upper()
            if overwrite_check == 'Y':
                #remove existing data
                data = [ a for a in data if a[0]!=car_name ]
            else:
                continue
            
        initial_pos = input(f"Please enter initial position of car {car_name} in x y Direction format:\n")

        # to ensure position given are in the correct format
        if re.fullmatch(r"\d+ \d+ [nsewNSEW]", initial_pos):

            # to ensure position given are not already occupied
            if initial_pos in [a[1] in data]:
                print("Location already occupied. Please start at a new location")
                continue

            # to ensure position given are within the initial boundary     
            xi, yi, di = initial_pos.split()
            if int(xi) <= num1 and int(yi) <= num2 and int(xi)>=0 and int(yi) >=0 :
                direction = input(f"Please enter the commands for car {car_name}:\n").strip()
            else: 
                print(f"Impossible initial position. The simulation field size is {num1} x {num2}. Please try again.\n")
                continue

            # to ensure comands given are in correct format
            if re.fullmatch(r"[lrfLRF]+", direction):
                return [car_name, f"({xi},{yi}) {di.upper()}", direction.upper()]
            else:
                print("Error. Please try again in the correct format.\n")
                
        else:
            print("Error. Please try again in the correct format for the 3 variables - x y direction(NSEW only) .\n")


# test2 = car_start(5,5,[['a',4,6],['b',1,2]])                           

In [5]:
def move_forward(direction,curr_x, curr_y):
    # Move the object forward by the given distance.
    radian_angle = math.radians(direction)
    curr_x +=  math.sin(radian_angle)
    curr_y +=  math.cos(radian_angle)
    return round(curr_x), round(curr_y)
    
def rotate_left(data):
    # Rotate the object left (counter-clockwise) by the given angle in degrees.
    return (data - 90)

def rotate_right(data):
    # Rotate the object right (clockwise) by the given angle in degrees.
    return (data + 90)


In [6]:
def clashing_cars(pnt, data):
    # check if any cars having matching location
    pnt_tmp = pnt[1].split()[0]
    data_tmp = [ [a[0], a[1].split()[0]] for a in data ]
    all_position = [ a[0] for a in data_tmp if a[1] == pnt_tmp and a[0] != pnt[0]]

    #to return False if list is empty, else return the list of collided cars
    if not all_position :
        return False 
    else:
        return all_position

In [55]:
def run_simulation(data, x, y):
    max_len = max([len(a[2]) for a in data])
    direction = {'N':0, 'E':90, 'S':180, 'W':270}
    
    # to loop through all possible commands 
    for a in range(max_len):
        
        # to loop through all cars
        for index, b in enumerate(data):

            # to check if it's final position clashes with any existing car position in board
            check = clashing_cars(b,data)
            if check:
                # collided cars will be appended to the end of list for tracking 
                b[4]=check
                # print(b)
                continue
            # print(b)
            #to skip the loop if it went out of range
            if len(b[2]) < a+1:
                continue
                      
            #extract current position and direction from list
            curr_pos, curr_dir = b[1].split()
            digits = re.findall(r'\d+', curr_pos)
            curr_x, curr_y = list(map(int, digits))

            # calculation
            # to rotate or move base on the command
            if b[2][a] =='L':
                tmp = rotate_left(direction[curr_dir])
                curr_dir = [k for k, v in direction.items() if v == tmp][0]
            elif b[2][a] == 'R':
                tmp = rotate_right(direction[curr_dir])
                curr_dir = [k for k, v in direction.items() if v == tmp][0]
            else: 
                curr_x, curr_y =  move_forward(direction[curr_dir], curr_x, curr_y)
                
            # to check if it's final position exceeds board boundary
            if curr_x > x or curr_y > y or curr_x < 0 or curr_y < 0:
                continue

            # update car new position into the list 
            b[1] = f"({curr_x},{curr_y}) {curr_dir}"
            
            #update loop
            b[3] +=1
    
    # output the result
    for a in data:
        if a[4] == '0':
            print(f"{a[0]}, {a[1]}, {a[3]} ,{a[4]}")
        else: 
            location = a[1].split()[0]
            col_cars = ", ".join(a[4])
            print(f"{a[0]} , collided with {col_cars} at {location} at step {a[3]}")
    

In [56]:
first = ['A', '(1,2) N', 'FFRFFFFRRL']
second = ['B', '(7,8) W', 'FFLFFFFFFF']
third = ['C','(5,2) E', 'LFFLRLRL']
fourth = ['D','(5,4) W', 'LRLRLRLR']

In [57]:
test = [first,second, third, fourth]
test = [ b + [0]+['0'] for b in test ]
run_simulation(test,10,10)
# clashing_cars(first,test)

A , collided with B, C, D at (5,4) at step 7
B , collided with A, C, D at (5,4) at step 7
C , collided with A, B, D at (5,4) at step 3
D , collided with A, B, C at (5,4) at step 2


In [291]:
def main():
    num1, num2 = field_check()
    registry = []
    loop1 = True
    loop2 = True
    while loop1 :
        selection = main_menu()
        
        if selection == 1:
            registry.append(car_start(num1,num2, registry))
        else: 
            #to add a marker tracking steps of car during looping and collision
            #registry format : [ 'car name','car position', 'command', 'step','collided cars' ]
            registry = [ b + [0] + ['0'] for b in registry ]
            result = run_simulation(registry, num1, num2)

            while loop2:
                check = input('''
                \nPlease choose from the following options:
                \n[1] Start over
                \n[2] Exit\n''').strip()
    
                if check =='1':
                    registry = []
                    loop2 = False 
    
                elif check =='2':
                    loop1 = False
                    loop2 = False 
                else:
                    print("Please enter either [1] or [2] only.")

            loop2 = True



In [292]:
main()

Welcome to Auto Driving Car Simulation!
        
Please enter the width and height of the simulation field in x y format:
 １０　１０


Invalid input. Please enter exactly two integers separated by a space.


Welcome to Auto Driving Car Simulation!
        
Please enter the width and height of the simulation field in x y format:
 10 10


You have created a field of 10 x 10.



        
Please choose from the following options:
        
[1] Add a car to field
        
[2] Run simulation
 1
Please input the name of the car:
 a
Please enter initial position of car a in x y Direction format:
 1 2 n
Please enter the commands for car a:
 frfrl

        
Please choose from the following options:
        
[1] Add a car to field
        
[2] Run simulation
 2


...


                
Please choose from the following options:
                
[1] Start over
                
[2] Exit
 1

        
Please choose from the following options:
        
[1] Add a car to field
        
[2] Run simulation
 1
Please input the name of the car:
 a
Please enter initial position of car a in x y Direction format:
 1 2 n
Please enter the commands for car a:
 fffff

        
Please choose from the following options:
        
[1] Add a car to field
        
[2] Run simulation
 20



Please choose either [1] or [2] only



        
Please choose from the following options:
        
[1] Add a car to field
        
[2] Run simulation
 2


a, (1,7) N



                
Please choose from the following options:
                
[1] Start over
                
[2] Exit
 2
